# Кратко об этом исследовании
Данные из Остинского центра животных, то есть приюта, -- с 1 октября 2013 по **март 2016**. 

## Цель

Требуется предсказать судьбу каждого животного по данным о нём сведениям. По сути, обычная задача категоризации. Классы: Adoption, Died, Euthanasia, Return to owner, Transfer. 

Все классы считаем одинаково важными все зависимости от представленности в выборке. Качество предсказаний оценивается поэтому с помощью macro-averaged F1 score.

## Методы
### Предобработка

**Имена**

С ними не удобно возиться, там много отсутствующих данных и их использование не оказывает явного влияния. Так что были удалены.

**Даты и возраст**

Помимо предложеного варианта, пробовал учитывать только год из DataTime как категориальный признак (значительной разницы не дает, а порой даже делает хуже). Пробовал несколько разных скейлеров (нормализаторов?), таких как StandartScaler и MinMaxScaler.

**Порода и цвет**

Пробовал разные энкодеры на категориальных признаках, а так же векторизацию. Идейно ожидал, что именно векторизация (с последующим применением энкодера или без него) даст хорошую предобработку для породы и цвета, однако на практике ничего хорошего почему-то не выходило. Пробовал в разных комбинациях учитывать и не учитывать эти признаки.

**Пол**

Пробовал делить и не делить на факт стерилизованности и пол отдельно. В среднем лучше себя показал раздельный вариант.

**Moreover**

Пробовал сбалансировать распределение меток у df_train с помощью догенерации данных алгоритмами SMOTE, ADASYN. Выглядело интересно, но результата не принесло. Возможно, при таком изначальном соотношении меток, на правильность предсказания редких меток (3, 4) проще не обращать внимания.

В целом для каждой исследуемой модели испытывались разные комбинации предобработки, параметров, энкодеров, векторизации, нормализации. Самые лучшеи результаты обычно выходили для комбинации OrdinalEncoder + StandartScaler без лишних наворотов.

### Модели

**LogisticRegression**: Ничего хорошего не вышло, но я не сильно и пытался - кажется эта модель не очень подходит по своей сути.

**DecisionTreeClassifier**: Пробовал на более менее стандартных параметрах, быстро перешел на леса.

**RandomForestClassifier**: Самая удачная модель в итоге. Сделал N запусков GridSearch на ней, перебираемые параметры будут ниже в коде.

**GradientBoostingClassifier**: Пробовал, но быстро перешел на реализацию XGBClassifier этой модели, так как где-то вычитал, что она реализована лучше, чем в Sklearn.

**XGBClassifier**: Вторая по удачности модель после RFC, но доститчь на ней высоких результатов так и не вышло. Сделал N запусков GridSearch на ней, перебираемые параметры будут ниже в коде. В целом эта модель была главным источником зря потраченных усилий.

**VotingClassifier**: То, что делается в итоге для RFC с разными параметрами. Действительно помогает. Пробовал заставить голосовать разные модели (в основном RFC с разными параметрами) в разном количестве. Менял параметр voting="soft"/"hard".


## Результаты

Про успехи и потраченные силы, кажется, было напсиано выше. В целом кажется, что для такого набора данных, с такими признаками и распределнием меток, лучше всего подходят логические классификаторы, основанные на построении деревьев. 

Смотря на итоговый рейтинг, кажется, что я уперся в некий потолок, где нужна была более глубокая предобработка, а не перебор параметров у моделей. В целом можно было бы вручную поработать с данными, поискать побольше корреляций разных параметров с метками на df_train, построить разные графики. Возможно, стоило бы вручную тщательно поработать с породами и цветами, перегруппировать эти данные, разбить на более широкие классы. Я никогда до этого не занимался ничем похожим на такое задание, так что в качестве главной морали я вынес то, что по сути надо проводить целое исследование о том, как устроены конкретные данные.
В общем интересно, что делали коллеги, получившие счет больше 0.5. Будет очень полезно, если вы после проверки работ вышлете в чат какую-то сводку по тому, за счет чего у людей получалось сделать хорошо.

## Конфиги и константы

In [262]:
OUTCOME2LABEL = {"Adoption" : 0, 
                 "Transfer": 1, 
                 "Return_to_owner": 2, 
                 "Euthanasia": 3, 
                 "Died": 4
                }
LABEL2OUTCOME = {v: k for k,v in OUTCOME2LABEL.items()}
FOLD_K = 4

## Библиотеки

In [263]:
! pip install xgboost

In [264]:
# Вроде все версии используемых библиотек стояли последние 
import sklearn
import pandas as pd
import numpy as np

from scipy import sparse 
from sklearn import tree 
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier
from xgboost import XGBClassifier

## Загрузка данных

In [265]:
df_train = pd.read_csv("train.csv", encoding="utf-8")
df_test = pd.read_csv("test.csv", encoding="utf-8")

## Удаление строк, где не хватает данных

Удаляем имена, так как это довольно бесполезный параметр. Удаляем строки df_train, где не хватет каких-то данных (15 штук без имен, не значительная потеря). В df_test заменяем пустые места на "Unknown".

In [266]:
df_train.drop(["Name"], axis=1, inplace=True)
df_train.dropna(inplace=True)

df_test.drop(["Name"], axis=1, inplace=True)
df_test.fillna("Unknown", inplace=True)

## Подготовка признаков

### Даты

In [267]:
def pandas_dates2number(date_series: pd.Series):
    return pd.to_datetime(date_series).values.astype(np.int64) // 10 ** 9

### Возраст

Преобразуем возраст к возрасту в днях. Для тех, у кого возраст исходно неизвестен, поставим среднее значение по известным, примерно равное 794 дням (для этого у "Unknown" сначала было выставлено значение в 0 дней и посчитано среднее по остальным).

In [268]:
# Функция, "разбирающая" строки с возрастом 
def age_to_days(age): 
    age_split = age.split()
    if age == "Unknown":
        age_in_days = 794
    elif age_split[1].startswith("year"):
        age_in_days = int(age_split[0]) * 365
    elif age_split[1].startswith("month"):
        age_in_days = int(age_split[0]) * 30
    elif age_split[1].startswith("week"):
        age_in_days = int(age_split[0]) * 7
    elif age_split[1].startswith("day"):
        age_in_days = int(age_split[0])
    return age_in_days

# Функция-преобразователь для DataFrame:
def Age_to_days_series(df: pd.Series):
    df = df.apply(lambda x: age_to_days(x))
    return df

### Пол

Разбиваем пол на стерилизованность и, собственно, пол.

In [269]:
# Функция выделения из строки факта стерилизованности
def intactness(sex_and_intact: str):
    if "Intact" in sex_and_intact:
        return "Intact"
    elif "Neutred" or "Spayed" in sex_and_intact:
        return "N or S"  # Neutred or Spayed
    return "Unknown"

# Функция выделения из строки пола
def sex(sex_and_intact: str):
    if "Male" in sex_and_intact:
        return "Male"
    elif "Female" in sex_and_intact:
        return "Female"
    return "Unknown"

# Функция-преобразователь для DataFrame для факта стерилизации:
def SexuponOutcome_to_intactness(df: pd.Series):
    df = df.apply(lambda x: intactness(x))
    return df

# Функция-преобразователь для DataFrame для пола:
def SexuponOutcome_to_sex(df: pd.Series):
    df = df.apply(lambda x: sex(x))
    return df

### Порода, цвет и тип животного

Будут напрямую обрабатываться через OrdinalEncoder

## Единая матрица признаков

In [270]:
def prepare_features(df, encoder=None, scaler=None):
    
    # Используем написанные ранее функции для преобразования признаков "DateTime", "SexuponOutcome", "AgeuponOutcome"
    df["DateTime"] = pandas_dates2number(df["DateTime"])  # Время
    
    df["intactness"] = SexuponOutcome_to_intactness(df["SexuponOutcome"])  # Факт стерилизованности
    df["Sex"] = SexuponOutcome_to_sex(df["SexuponOutcome"])  # Пол
    df.drop(["SexuponOutcome"], axis=1, inplace=True)
    
    df["Age"] = Age_to_days_series(df["AgeuponOutcome"])  # Возраст
    df.drop(["AgeuponOutcome"], axis=1, inplace=True)
    

    columns_categorical = ["AnimalType", "Color", "Breed", "intactness", "Sex"]
    columns_real = ["DateTime", "Age"]
    
    
    # Для columns_categorical используем OrdinalEncoder
    if encoder is None:
        encoder = OrdinalEncoder(handle_unknown="use_encoded_value", unknown_value=-1)
        encoder.fit(df[columns_categorical])

    encoded_categorical_features = encoder.transform(df[columns_categorical])
    cat_feature_names = encoder.categories_   # Новые названия категориальных колонок
    
    # Для columns_real используем StandardScaler
    if scaler is None:
        scaler = StandardScaler()
        scaler.fit(df[columns_real])

    scaled_real_features = scaler.transform(df[columns_real])

    # Соединяем обработанные признаки в единую матрицу X
    X = np.hstack([encoded_categorical_features, scaled_real_features]) 
    
    # Передаем матрицу X, новые названия колонок (не знаю зачем) и 
    # "правильные" encoder и scaler для использования при преобразовании df_test
    return X, list(cat_feature_names) + ["DateTime", "Age"], encoder, scaler

## Создание матриц для классификации

In [271]:
X_train, fnames, encoder, scaler = prepare_features(df_train)
y_train = df_train["Outcome"]
X_test, _, _, _ = prepare_features(df_test, encoder, scaler)

## GridSearch

### Параметры для GridSearch

Конечно, с таким количеством параметров перебор шел бы слишком долго, поэтому в реальности запускалось много разных более "узких" GridSearch или использовался метод постепенного уточнения/сужения разброса параметров в последовательных GridSearch. Так что здесь просто представлены самые полные матрицы перебираемых параметров.

In [272]:
params_for_RFC = {
    "n_estimators": [50, 100, 150, 200, 250, 300, 400, 500],
    "max_features": ["sqrt", "log2"],
    "max_depth": [None, 10, 20, 30],
    "max_samples": [None, 0.1, 0.2, 0.3, 0.4, 0.5],
    "min_samples_leaf": [2, 3, 5, 8, 10],
    "min_samples_split": [2, 3, 5, 8, 10],
    "class_weight": [None, "balanced_subsample"],
    "bootstrap": [True]}
# И примерно то же самое для RFC при bootstrap=False (В таком случае max_samples=None и class_weight=None/"balanced")

# Перебирались также такие парамтеры для XGBClassifier, но это не дало значительного улучшения
params_for_XGB = {
    "n_estimators": [200, 400, 800, 1000],
    "learning_rate": [0.03, 0.07, 0.1],
    "max_depth": [2, 4, 6, 8],
    "objective": ["multi:softprob"],
    "colsample_bytree": [0.4, 0.6, 0.8, 1],
    "reg_lambda": [0.1, 0.3, 1, 3]}

### Пример производимого GridSearch

In [273]:
# Пример производимого GridSearch

"""
scores = ["f1_macro", "f1_micro", "neg_log_loss"]  # На самом деле по наблюдениям можно использовать только "f1_macro"

for score in scores:
    
    print("# Tuning for %s" % score)
    print()

    cv = StratifiedKFold(n_splits=2, random_state=1, shuffle=True)
    clf_RFC = RandomForestClassifier(n_jobs=-1, random_state=1)
    
    # поиск по заданной решётке параметров
    clf_grid = GridSearchCV(clf_RFC, params_for_RFC, scoring=score, n_jobs=-1, cv=cv, verbose=2)
    
    # запускаем поиск
    clf_grid.fit(X_train, y_train)
    
    # Вывод лучших парметров
    print("Best params on dev set:")
    print(clf_grid.get_params())
    print()
    
    # обучаем на всём с "лучшими" параметрами
    best_RFC = clf_grid.best_estimator_
    best_RFC.fit(X_train, y_train)
    y_pred = best_RFC.predict(X_test)
    
    # Запись полученных данных
    pd.DataFrame({"ID": df_test["ID"], "Outcome": y_pred}).to_csv("submission_" + score + ".csv", index=None)
"""    

'\nscores = ["f1_macro", "f1_micro", "neg_log_loss"]  # На самом деле по наблюдениям можно использовать только "f1_macro"\n\nfor score in scores:\n    \n    print("# Tuning for %s" % score)\n    print()\n\n    cv = StratifiedKFold(n_splits=2, random_state=1, shuffle=True)\n    clf_RFC = RandomForestClassifier(n_jobs=-1, random_state=1)\n    \n    # поиск по заданной решётке параметров\n    clf_grid = GridSearchCV(clf_RFC, params_for_RFC, scoring=score, n_jobs=-1, cv=cv, verbose=2)\n    \n    # запускаем поиск\n    clf_grid.fit(X_train, y_train)\n    \n    # Вывод лучших парметров\n    print("Best params on dev set:")\n    print(clf_grid.get_params())\n    print()\n    \n    # обучаем на всём с "лучшими" параметрами\n    best_RFC = clf_grid.best_estimator_\n    best_RFC.fit(X_train, y_train)\n    y_pred = best_RFC.predict(X_test)\n    \n    # Запись полученных данных\n    pd.DataFrame({"ID": df_test["ID"], "Outcome": y_pred}).to_csv("submission_" + score + ".csv", index=None)\n'

## Лучшие параметры

На самом деле, лучшие параметры выходили разными на разных запусках GridSearch (со слегка разным перебором) и они давали очень близкие оценки на Public leaderboard, поэтому я взял некоторые лучшие и устроил голосование классификаторов.

In [274]:
# Некоторые лучшие при bootstrap=True
params_for_RFC1 = {
    "n_estimators": [100, 150, 200, 250, 300, 400],  # Бывает удачно при довольно разных n_estimators
    "max_features": ["sqrt"],
    "max_depth": [20, 30],
    "max_samples": [0.3, 0.4],
    "min_samples_leaf": [5, 8],
    "min_samples_split": [2, 3, 5, 8, 10],  # Тут бывает любое из этих значений, не сильно влияет на счет
    "class_weight": ["balanced_subsample"],
    "bootstrap": [True]}

# Некоторые лучшие при bootstrap=False
params_for_RFC2 = {
    "n_estimators": [100, 150, 200],  # Бывает удачно при довольно разных n_estimators, но в среднем стали поменьше, чем с True
    "max_features": ["sqrt"],
    "max_depth": [None, 20, 30],
    "min_samples_leaf": [3, 5],
    "min_samples_split": [2, 3, 5, 8, 10],  # Тут бывает любое из этих значений, не сильно влияет на счет
    "class_weight": ["balanced"],
    "bootstrap": [False]}

# XGBoosting. На самом деле лучшие результаты он показывал при всех дефолтных параметрах, 
# разве что objective="multi:softprob" указать полезно, что и отражено во варианте, используемом при голосовании

## VotingClassifier

### Классификаторы для Voting

Я не придумал, как задать все эти классификаторы красиво.

XGBoosting был включен главным образом потому, что я потратил на попытки довести его до ума несколько дней, а потом RFC оказались лучше, но мне все таки хотелось его куда-нибудь запихнуть)

Также, смотря на итоговые баллы для разных сабмитов, стоит отметить, что лучший вариант был не при таком относительно хаотичном голосовании 8 моделей, а при голосовании 2 деревьев и XGB (но вообще это все похоже на "как повезет").

In [275]:
# RandomForestClassifie с bootstrap=True
clf1 = RandomForestClassifier(bootstrap=True, class_weight="balanced_subsample", 
                              max_depth=20, max_samples=0.3, min_samples_leaf=5, 
                              min_samples_split=2, n_estimators=100, n_jobs=-1,
                              random_state=1)

clf2 = RandomForestClassifier(bootstrap=True, class_weight="balanced_subsample",
                              max_depth=20, max_samples=0.3, min_samples_leaf=8, 
                              min_samples_split=2, n_estimators=300, n_jobs=-1, 
                              random_state=1)

clf3 = RandomForestClassifier(bootstrap=True, class_weight="balanced_subsample", 
                              max_depth=20, max_samples=0.4, min_samples_leaf=8, 
                              min_samples_split=10, n_estimators=250, n_jobs=-1, 
                              random_state=1)

clf4 = RandomForestClassifier(bootstrap=True, class_weight="balanced_subsample", 
                              max_depth=30, max_samples=0.4, min_samples_leaf=5, 
                              min_samples_split=2, n_estimators=400, n_jobs=-1, 
                              random_state=1)

# RandomForestClassifie с bootstrap=False
clf5 = RandomForestClassifier(bootstrap=False, class_weight="balanced", 
                              max_depth=None, max_samples=None, min_samples_leaf=5, 
                              min_samples_split=10, n_estimators=100, n_jobs=-1, 
                              random_state=1)

clf6 = RandomForestClassifier(bootstrap=False, class_weight="balanced", 
                              max_depth=20, max_samples=None, min_samples_leaf=3, 
                              min_samples_split=3, n_estimators=100, n_jobs=-1, 
                              random_state=1)

clf7 = RandomForestClassifier(bootstrap=False, class_weight="balanced", 
                              max_depth=30, max_samples=None, min_samples_leaf=5,
                              min_samples_split=5, n_estimators=150, n_jobs=-1, 
                              random_state=1)

# XGBoosting
clf_xgb = XGBClassifier(objective='multi:softprob',n_jobs=-1,seed=1)

## Voting

In [276]:
clfs = [clf1, clf2, clf3, clf4, clf5, clf6, clf7, clf_xgb]
clfs_names = ["clf1", "clf2", "clf3", "clf4", "clf5", "clf6", "clf7", "clf_xgb"]
clfs_zip = list(zip(clfs_names, clfs))  # Создаем список вида [(" " , ),(" " , ),...] 

clf_vot = VotingClassifier(estimators=clfs_zip, voting="soft", n_jobs=-1) 
clf_vot.fit(X_train, y_train)
y_pred = clf_vot.predict(X_test)

# Записываем полученные данные в файл
pd.DataFrame({"ID": df_test["ID"], "Outcome": y_pred}).to_csv("submission_voting" + ".csv", index=None)